In [1]:
import Model
import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
import os
import re
import hashlib
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
import SpeechGenerator
import librosa
#from keras import losses

from extractMFCC import computeFeatures, computeFeatures1
from addNoise import addNoise
print(tf.__version__)

Using TensorFlow backend.


1.14.0


In [2]:
# There is a problem with the maximum lenght of a string then We save the Dataset root folder as a variable
dataset_dir = "Dataset/"

In [3]:
testWAVs = pd.read_csv(dataset_dir + 'testing_list.txt', sep=" ", header=None)[0].tolist()
valWAVs  = pd.read_csv(dataset_dir + 'validation_list.txt', sep=" ", header=None)[0].tolist()
"""
for idx, item in enumerate(testWAVs):
    testWAVs[idx] = "Dataset/" + testWAVs[idx]

for idx, item in enumerate(valWAVs):
    valWAVs[idx] = "Dataset/" + valWAVs[idx]
"""

'\nfor idx, item in enumerate(testWAVs):\n    testWAVs[idx] = "Dataset/" + testWAVs[idx]\n\nfor idx, item in enumerate(valWAVs):\n    valWAVs[idx] = "Dataset/" + valWAVs[idx]\n'

In [4]:
DictCategs = {'nine' : 1, 'yes' : 2, 'no' : 3, 'up' : 4, 'down' : 5, 'left' : 6, 'right' : 7, 'on' : 8, 'off' : 9, 
              'stop' : 10, 'go' : 11, 'zero' : 12, 'one' : 13, 'two' : 14, 'three' : 15, 'four' : 16, 'five' : 17, 
              'six' : 18, 'seven' : 19, 'eight' : 20, 'backward':0, 'bed':0, 'bird':0, 'cat':0, 'dog':0, 'follow':0, 
              'forward':0, 'happy':0, 'house':0, 'learn':0, 'marvin':0, 'sheila':0, 'tree':0, 'visual':0, 'wow':0 }
nCategs = 21

In [5]:
allWAVs  = []
for root, dirs, files in os.walk('Dataset/'):
    for f in files:
        if (root != dataset_dir + "_background_noise_") and (f.endswith('.wav')):
            path = root + "/" + f
            #print(path)
            path = path[len(dataset_dir):]
            #print(path)
            allWAVs.append(path)
    """
    print(f for f in files if f.endswith('.wav'))
    if root != dataset_dir + "_background_noise_":
        allWAVs += [root+'/'+ f for f in files if f.endswith('.wav')]
    """
trainWAVs = list(set(allWAVs) - set(valWAVs) - set(testWAVs))

In [6]:
print(len(trainWAVs))
print(len(valWAVs))
print(len(testWAVs))

84843
9981
11005


In [7]:
def _getFileCategory(file, catDict):
    # Receives a file with name <cat>/<filename> and returns an integer that is catDict[cat]
    categ = os.path.basename(os.path.dirname(file))
    return catDict.get(categ, 0)

In [8]:
#get categories
testWAVlabels = [_getFileCategory(f, DictCategs) for f in testWAVs]
valWAVlabels = [_getFileCategory(f, DictCategs) for f in valWAVs]
trainWAVlabels = [_getFileCategory(f, DictCategs) for f in trainWAVs]

In [9]:
print(len(trainWAVlabels))
print(len(valWAVlabels))
print(len(testWAVlabels))

84843
9981
11005


In [10]:
"""def load_and_preprocess_data(file_name):
    # Required by tensorflow (strings are passed as bytes)
    # tf.strings.as_string(...)
    file_name = file_name.numpy()
    file_name = file_name.decode("utf-8") 
    #print(file_name)
    #print(type(file_name))

    # Load data
    feats = computeFeatures(file_name)
    # Normalize
    mean = np.mean(feats, axis = 0)
    stv = np.std(feats, axis = 0)
    diff = np.subtract(feats, mean)
    feats = np.divide(diff, stv)

    return feats.astype(np.float32)

# example:

feats = load_and_preprocess_data(trainWAVs[25])
feats = np.transpose(feats)

plt.figure(figsize=(17,6))
plt.pcolormesh(feats)

plt.title('Spectrogram visualization')
plt.ylabel('Frequency')
plt.xlabel('Time')

plt.show()
print(trainWAVlabels[25])
"""

'def load_and_preprocess_data(file_name):\n    # Required by tensorflow (strings are passed as bytes)\n    # tf.strings.as_string(...)\n    file_name = file_name.numpy()\n    file_name = file_name.decode("utf-8") \n    #print(file_name)\n    #print(type(file_name))\n\n    # Load data\n    feats = computeFeatures(file_name)\n    # Normalize\n    mean = np.mean(feats, axis = 0)\n    stv = np.std(feats, axis = 0)\n    diff = np.subtract(feats, mean)\n    feats = np.divide(diff, stv)\n\n    return feats.astype(np.float32)\n\n# example:\n\nfeats = load_and_preprocess_data(trainWAVs[25])\nfeats = np.transpose(feats)\n\nplt.figure(figsize=(17,6))\nplt.pcolormesh(feats)\n\nplt.title(\'Spectrogram visualization\')\nplt.ylabel(\'Frequency\')\nplt.xlabel(\'Time\')\n\nplt.show()\nprint(trainWAVlabels[25])\n'

In [11]:
"""def create_dataset(file_names, labels, batch_size, shuffle, cache_file=None):
    
    # Create a Dataset object
    dataset = tf.data.Dataset.from_tensor_slices((file_names, labels))
    
    # Map the load_and_preprocess_data function
    py_func = lambda file_name, label: (tf.py_function(load_and_preprocess_data, [file_name], Tout=tf.float32), label)
    dataset = dataset.map(py_func, num_parallel_calls=os.cpu_count())
    
    # Cache dataset
    if cache_file:
        dataset = dataset.cache(cache_file)
    
    # Shuffle    
    if shuffle:
        dataset = dataset.shuffle(len(file_names))
        
    # Repeat the dataset indefinitely
    dataset = dataset.repeat()
    
    # Correct input shape for the network
    dataset = dataset.map(lambda data, label: (tf.expand_dims(data, 1), label))
    
    # Batch
    dataset = dataset.batch(batch_size=batch_size)
    
    # Prefetch
    dataset = dataset.prefetch(buffer_size=1)
    
    return dataset
"""

'def create_dataset(file_names, labels, batch_size, shuffle, cache_file=None):\n    \n    # Create a Dataset object\n    dataset = tf.data.Dataset.from_tensor_slices((file_names, labels))\n    \n    # Map the load_and_preprocess_data function\n    py_func = lambda file_name, label: (tf.py_function(load_and_preprocess_data, [file_name], Tout=tf.float32), label)\n    dataset = dataset.map(py_func, num_parallel_calls=os.cpu_count())\n    \n    # Cache dataset\n    if cache_file:\n        dataset = dataset.cache(cache_file)\n    \n    # Shuffle    \n    if shuffle:\n        dataset = dataset.shuffle(len(file_names))\n        \n    # Repeat the dataset indefinitely\n    dataset = dataset.repeat()\n    \n    # Correct input shape for the network\n    dataset = dataset.map(lambda data, label: (tf.expand_dims(data, 1), label))\n    \n    # Batch\n    dataset = dataset.batch(batch_size=batch_size)\n    \n    # Prefetch\n    dataset = dataset.prefetch(buffer_size=1)\n    \n    return dataset\n'

In [12]:
"""batch_size = 32
train_dataset = create_dataset(trainWAVs, 
                               trainWAVlabels, 
                               batch_size=batch_size, 
                               shuffle=True,
                               cache_file='train_cache')

val_dataset = create_dataset(valWAVs, 
                             valWAVlabels,
                             batch_size=batch_size, 
                             shuffle=False,
                             cache_file='val_cache')

train_steps = int(np.ceil(len(trainWAVs)/batch_size))
val_steps = int(np.ceil(len(valWAVs)/batch_size))
"""

"batch_size = 32\ntrain_dataset = create_dataset(trainWAVs, \n                               trainWAVlabels, \n                               batch_size=batch_size, \n                               shuffle=True,\n                               cache_file='train_cache')\n\nval_dataset = create_dataset(valWAVs, \n                             valWAVlabels,\n                             batch_size=batch_size, \n                             shuffle=False,\n                             cache_file='val_cache')\n\ntrain_steps = int(np.ceil(len(trainWAVs)/batch_size))\nval_steps = int(np.ceil(len(valWAVs)/batch_size))\n"

In [13]:
# Transoform the dataset in numpy array and load 
train = np.array(trainWAVs, dtype = object)
trainLabels = np.array(trainWAVlabels, dtype = '>i4') #stands for int32

print("Loading of the train set:")
for i in range(len(trainWAVs)):
    # Print the progress 
    if (i % 5000) == 0:
        print(str(i) + '/' + str(len(trainWAVs)))
    
    # If the file is not already present, we create the numpy version 
    if (not os.path.isfile(dataset_dir + "/" + trainWAVs[i] + '.npy')):
        y, sr = librosa.load(dataset_dir + "/" + trainWAVs[i], sr = 16000)
        np.save(dataset_dir + "/" + trainWAVs[i] + '.npy', y)
    
    # We load the path to numpy array in a vector 
    train[i] = trainWAVs[i] + '.npy'
    
print(str(i+1) + '/' + str(len(trainWAVs)))

Loading of the train set:
0/84843
5000/84843
10000/84843
15000/84843
20000/84843
25000/84843
30000/84843
35000/84843
40000/84843
45000/84843
50000/84843
55000/84843
60000/84843
65000/84843
70000/84843
75000/84843
80000/84843
84843/84843


In [14]:
# Do the same thing for the validation and the test set
val = np.array(valWAVs, dtype = object)
valLabels = np.array(valWAVlabels, dtype = '>i4') #stands for int32

print("Loading of the validation set:")
for i in range(len(valWAVs)):
    # Print the progress 
    if (i % 5000) == 0:
        print(str(i) + '/' + str(len(valWAVs)))
    
    # If the file is not already present, we create the numpy version 
    if (not os.path.isfile(dataset_dir + "/" + valWAVs[i] + '.npy')):
        y, sr = librosa.load(dataset_dir + "/" + valWAVs[i], sr = 16000)
        np.save(dataset_dir + "/" + valWAVs[i] + '.npy', y)
    
    # We load the path to numpy array in a vector 
    val[i] = valWAVs[i] + '.npy'
    
print(str(i+1) + '/' + str(len(valWAVs)))

test = np.array(testWAVs, dtype = object)

print("Loading of the test set:")
for i in range(len(testWAVs)):
    # Print the progress 
    if (i % 5000) == 0:
        print(str(i) + '/' + str(len(testWAVs)))
    
    # If the file is not already present, we create the numpy version 
    if (not os.path.isfile(dataset_dir + "/" + testWAVs[i] + '.npy')):
        y, sr = librosa.load(dataset_dir + "/" + testWAVs[i], sr = 16000)
        np.save(dataset_dir + "/" + testWAVs[i] + '.npy', y)
    
    # We load the path to numpy array in a vector 
    test[i] = testWAVs[i] + '.npy' 
    
print(str(i+1) + '/' + str(len(testWAVs)))

Loading of the validation set:
0/9981
5000/9981
9981/9981
Loading of the test set:
0/11005
5000/11005
10000/11005
11005/11005


In [15]:
#test on the length
print(str(len(trainWAVs)) + " = " + str(len(train)))
print(str(len(valWAVs)) + " = " + str(len(val)))
print(str(len(testWAVs)) + " = " + str(len(test)))

#test on labels 
for i in range(0, 25):
    print("file: " + train[i] + " - label: " + str(trainLabels[i]))
    print("file: " + trainWAVs[i] + " - label: " + str(trainLabels[i]))

84843 = 84843
9981 = 9981
11005 = 11005
file: follow/f035e2ea_nohash_3.wav.npy - label: 0
file: follow/f035e2ea_nohash_3.wav - label: 0
file: dog/3ab9ba07_nohash_1.wav.npy - label: 0
file: dog/3ab9ba07_nohash_1.wav - label: 0
file: two/1ed0b13d_nohash_1.wav.npy - label: 14
file: two/1ed0b13d_nohash_1.wav - label: 14
file: bird/30a09789_nohash_1.wav.npy - label: 0
file: bird/30a09789_nohash_1.wav - label: 0
file: two/b31ad508_nohash_0.wav.npy - label: 14
file: two/b31ad508_nohash_0.wav - label: 14
file: happy/1890e5c3_nohash_0.wav.npy - label: 0
file: happy/1890e5c3_nohash_0.wav - label: 0
file: bed/c6ca5d01_nohash_0.wav.npy - label: 0
file: bed/c6ca5d01_nohash_0.wav - label: 0
file: nine/c948d727_nohash_0.wav.npy - label: 1
file: nine/c948d727_nohash_0.wav - label: 1
file: seven/7213ed54_nohash_3.wav.npy - label: 19
file: seven/7213ed54_nohash_3.wav - label: 19
file: house/ad526ada_nohash_0.wav.npy - label: 0
file: house/ad526ada_nohash_0.wav - label: 0
file: go/94d370bf_nohash_0.wav.n

In [16]:
trainGen = SpeechGenerator.SpeechGen(train, trainLabels, dataset_dir, batch_size = 64, shuffle = True)
valGen = SpeechGenerator.SpeechGen(val, valLabels, dataset_dir, batch_size = 16, shuffle = False)    

In [20]:
#keras.backend.set_floatx('float32')

#print(keras.backend.floatx())

model = Model.SimpleModel(nCategs, 99, 39, use_GRU = True, dropout = 0.1, activation = 'relu')
"""
decayed_lr = tf.train.exponential_decay(0.0000001,
                                        0.8, 50,
                                        0.95, staircase=True)

adam = tf.train.AdamOptimizer(learning_rate=decayed_lr,  
                                beta1=0.9,
                                beta2=0.999,
                                epsilon=1e-07,
                                use_locking=False,
                                name='Adam')
"""
#adam = tf.train.AdamOptimizer(learning_rate=0.00000001,  
#                                beta1=0.9,
#                                beta2=0.999,
#                                epsilon=1e-07,
#                                use_locking=False,
#                                name='Adam')

#model.compile(optimizer=adam,
#              loss= tf.losses.softmax_cross_entropy,
#              metrics=[keras.metrics.sparse_categorical_accuracy, ])

model.compile(optimizer='adam', loss=['sparse_categorical_crossentropy'], metrics=['sparse_categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 99, 39, 1)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 95, 35, 64)        1664      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 31, 11, 64)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 31, 11, 64)        256       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 29, 9, 32)         18464     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 9, 3, 32)          0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 9, 3, 32)          128       
__________

In [21]:
import math
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.4
    epochs_drop = 3.0
    lrate = initial_lrate * math.pow(drop,  
            math.floor((1+epoch)/epochs_drop))
    
    if (lrate < 4e-5):
        lrate = 4e-5
      
    print('Changing learning rate to {}'.format(lrate))
    return lrate
lrate = LearningRateScheduler(step_decay)

earlystopper = EarlyStopping(monitor='val_categorical_accuracy', patience=10, verbose=1)
checkpointer = ModelCheckpoint('Model/my_model_loss_dropout-0_05-yes_reg-0.h5', 
                               monitor='val_categorical_accuracy', 
                               verbose=1, save_best_only = True, save_weights_only = True)

In [ ]:
num_epochs = 1
history = model.fit_generator(trainGen, epochs = num_epochs, callbacks = [earlystopper, checkpointer, lrate])

print(history.history.keys())
# Plot loss
plt.figure()
plt.plot(history.history['loss'], label='Train loss')
plt.plot(history.history['val_loss'], label='Val loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')

# Plot accuracy
plt.figure()
plt.plot(history.history['categorical_accuracy'], label='Train acc')
plt.plot(history.history['val_categorical_accuracy'], label='Val acc')
plt.legend()

plt.xlabel('Epoch')
plt.ylabel('Accuracy')

W0627 16:47:12.461753 140127912863488 deprecation.py:323] From /home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1
Changing learning rate to 0.001
 251/1325 [====>.........................] - ETA: 1:00:29 - loss: 2.3901 - sparse_categorical_accuracy: 0.3068

In [ ]:
#print(model.get_weights())
booo = np.array(model.get_weights())
for i in range(len(booo)):
    print(str(np.max(booo[i])) + " - min: " + str(np.min(booo[i]))) 

In [ ]:
#Debugging
attSpeechModel = keras.Model(inputs=model.input, outputs=[model.get_layer('output').output, 
                                                    model.get_layer('attSoftmax').output,
                                                    model.get_layer('mel_stft').output])

gradients = keras.backend.gradients(model.output, model.input)              #Gradient of output wrt the input of the model (Tensor)
print(gradients)

In [ ]:
audios, classes = valGen.__getitem__(1851)
print(classes)

In [ ]:
idAudio = 0
print(classes[idAudio])

In [ ]:
outs, attW, specs = attSpeechModel.predict( audios )


In [ ]:
boh = np.isnan(specs)
np.where(boh == True)
print(specs)

In [ ]:
# Save the model (weights only)
model.save_weights('Model/my_model_sparse-loss_dropout-0_1-yes_reg-0.h5')
# Load model (weights only)
#model = keras.models.load_weights("Model/")

In [ ]:
testGen = SpeechGenerator.SpeechGen(test, testWAVlabels, dataset_dir, shuffle=False)

testEval = model.evaluate_generator(testGen, verbose = 1)

print("loss: " + str(testEval[0]))
print("acc: " + str(testEval[1]))

  9/171 [>.............................] - ETA: 9:28

In [ ]:
# I don't now why i need to reshape in this way but it works 
x = np.load("Dataset/tree/022cd682_nohash_1.wav.npy").reshape((1, 16000))
print(x.shape)
res = model.predict(x) 
print(res.argmax())